In [ ]:
# default_exp splitting

# Splitting

> Simple functions to save time when creating validation and testing splits

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [ ]:
df = pd.DataFrame(np.random.randint(0,101,10000),columns=['dep_var'])
df.head()

,dep_var
0,60
1,45
2,50
3,87
4,97


In [ ]:
#export
def bin_df(df,col,bin_sz):
    if not (df[col].max() - df[col].min()) % bin_sz == 0:
        for new_sz in range(bin_sz,df[col].max()):
            if (df[col].max() - df[col].min()) % new_sz == 0:
                print(f'bin sizes not right - next largest size is {new_sz}')
                assert (df[col].max() - df[col].min()) % bin_sz == 0
    for i in range(df[col].min(),df[col].max()+bin_sz,bin_sz):
        bin_min = i
        bin_max = i + bin_sz - 1
        mask = (df[col] >= bin_min) & (df[col] <= bin_max)
        df.loc[mask,f'bin_{bin_sz}'] = f'{bin_min}-{bin_max}'
    return df

In [ ]:
df = bin_df(df,'dep_var',5)
df.head()

,dep_var,bin_5,bin_10
0,60,56-60,51-60
1,45,41-45,41-50
2,50,46-50,41-50
3,87,86-90,81-90
4,97,96-100,91-100


In [ ]:
#export
def kfold_Stratified_df(df,col,folds):
    skf = StratifiedKFold(n_splits=folds)
    col_name = f'{folds}fold_{col}'
    f_num = 0
    for train_idxs, valid_idxs in skf.split(df.index,df[col]):
        df.loc[df.index.isin(valid_idxs),col_name] = int(f_num)
        f_num += 1
    return df

In [ ]:
df = kfold_Stratified_df(df,'bin_5',3)
df.groupby('3fold_bin_5').count()

,dep_var,bin_5
3fold_bin_5,,
0.0,334,334
1.0,333,333
2.0,333,333


In [ ]:
df.groupby(['3fold_bin_5','bin_5']).count().reset_index().sort_values(['bin_5','3fold_bin_5']).head(9)

,3fold_bin_5,bin_5,dep_var
0,0.0,-4-0,3
22,1.0,-4-0,3
44,2.0,-4-0,3
1,0.0,1-5,16
23,1.0,1-5,16
45,2.0,1-5,16
2,0.0,101-105,3
24,1.0,101-105,3
46,2.0,101-105,3
